In [ ]:
#Importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import numpy as np
import os
import time

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from pathlib import Path
from collections import Counter
from sklearn.pipeline import Pipeline
from openpyxl import load_workbook

In [ ]:
#Reading the excel-document into a df
df = pd.read_excel('dataset_name.xlsx', Sheet_name = 'Sheet_name') 

#Choosing which columns to keep and saving them in a new df
columns_to_keep = ['Column 1', 'Column 2', 'Column 3', 'Column 4', 'Column 5', '...']
df2 = df[columns_to_keep].copy()

In [ ]:
#Splitting df into two based on hole ID
df2.loc[:, 'Hole_id_stat_str'] = df2['Hole_id_stat'].astype(str) #Making sure the IDs are strings

#Defining the unseen column(s)
columns_unseen = ['Column']
#Splitting into train and unseen data based on column
df_train = df2[~df2['Hole_id_stat_str'].isin(columns_unseen)] 
df_unseen = df2[df2['Hole_id_stat_str'].isin(columns_unseen)] 


In [ ]:
#Defining which columns to keep av model input
X = df_train.drop(columns = ['Column 1', 'Column 2', 'Column 3'])

#Defining which columns are the model outputs
y = df_train['Column 4']

#Defining inputs and outputs for unseen dataset
X_unseen = df_unseen.drop(columns = ['Column 1', 'Column 2', 'Column 3')
y_unseen = df_unseen['Column 4'] 

In [ ]:
#Setting up K-Fold cross-validation
k_fold = KFold(n_splits=5, random_state=66, shuffle=True)


# Defining a pipeline for scaler and RF regressor
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor(n_estimators=40, random_state=40))
])

#Creating lists to store performance metrics for each fold
mae_scores_train, mse_scores_train, r2_scores_train = [], [], []
mae_scores_test, mse_scores_test, r2_scores_test = [], [], []

#Looping over each fold in the K-Fold split
for train_index, test_index in k_fold.split(X, y):
    #Splitting the data into training and test subsets for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #Skipping fold if the test target values contain only one class
    if len(np.unique(y_test)) == 1:
        print("Skipping this fold due to only one class in test set.")
        continue

    #Fitting the pipeline on the training data (scaling + random forest training)
    pipeline.fit(X_train, y_train)

    #Making predictions on both training and test data
    pred_train = pipeline.predict(X_train)
    pred_test = pipeline.predict(X_test)

    #Evaluating performance on the training data
    mae_train = mean_absolute_error(y_train, pred_train)
    mse_train = mean_squared_error(y_train, pred_train)
    r2_train = r2_score(y_train, pred_train)

    #Evaluating performance on the test data
    mae_test = mean_absolute_error(y_test, pred_test)
    mse_test = mean_squared_error(y_test, pred_test)
    r2_test = r2_score(y_test, pred_test)

    #Storing performance metrics for this fold
    mae_scores_train.append(mae_train)
    mse_scores_train.append(mse_train)
    r2_scores_train.append(r2_train)

    mae_scores_test.append(mae_test)
    mse_scores_test.append(mse_test)
    r2_scores_test.append(r2_test)

    #Printing metrics for the current fold
    print(f"Train -> MAE: {mae_train:.6f}, MSE: {mse_train:.6f}, R²: {r2_train:.6f}")
    print(f"Test  -> MAE: {mae_test:.6f}, MSE: {mse_test:.6f}, R²: {r2_test:.6f}\n")

#Calculating and print the average performance across all valid folds
print("\nAverage result from all folds:")
print(f"Train Mean MAE: {np.mean(mae_scores_train):.6f}, Test Mean MAE: {np.mean(mae_scores_test):.6f}")
print(f"Train Mean MSE: {np.mean(mse_scores_train):.6f}, Test Mean MSE: {np.mean(mse_scores_test):.6f}")
print(f"Train Mean R²: {np.mean(r2_scores_train):.6f}, Test Mean R²: {np.mean(r2_scores_test):.6f}")


In [ ]:
#Defining a new unfitted pipeline 
pipeline = Pipeline([
    ('scaler', StandardScaler()),                 
    ('rf', RandomForestRegressor(random_state=66))  
])

#Defining the hyperparameter grid
parameters_grid = {
    'rf__n_estimators': [100],               
    'rf__max_features': [15],                
    'rf__max_depth': [7, 9, 11],            
    'rf__min_samples_split': [2],            
    'rf__min_samples_leaf': [2, 5],         
    'rf__criterion': ['squared_error'],     
}

#Setting up GridSearchCV to tune hyperparameters
CV_RF = GridSearchCV(
    estimator=pipeline,
    param_grid=parameters_grid,
    cv=k_fold,
    scoring='r2',
    n_jobs=-1
)

#Fitting the grid search on the entire dataset
CV_RF.fit(X, y)

#Printing the best combination of hyperparameters found during the search
print('Best parameters: ', CV_RF.best_params_)


In [ ]:
#Scaling X_train and X_test. Fitting and transforming on X_train and trensforming on X_test
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)  
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)         

#Extracting the best hyperparameters from GridSearchCV results
best_params = {key.replace("rf__", ""): value for key, value in CV_RF.best_params_.items()}

#Initializing a new RandomForestRegressor using the best parameters
RF_best = RandomForestRegressor(**best_params)

#Training the model on the scaled training data
RF_best.fit(X_train_scaled, y_train)

#Generating predictions on both the training and test sets
y_train_pred = RF_best.predict(X_train_scaled)
y_test_pred = RF_best.predict(X_test_scaled)

#Evaluating the model's performance on the training data
mse_train = mean_squared_error(y_train, y_train_pred)  
r2_train = r2_score(y_train, y_train_pred)              

#Evaluating the model's performance on the test data
mse_test = mean_squared_error(y_test, y_test_pred)     
r2_test = r2_score(y_test, y_test_pred)               

#Printing the evaluation metrics
print(f"Train Mean Squared Error: {mse_train:.4f}")
print(f"Train R-squared: {r2_train:.4f}")
print(f"Test Mean Squared Error: {mse_test:.4f}")
print(f"Test R-squared: {r2_test:.4f}")


In [ ]:
#Scaling the unseen features using the same scaler that was fitted on the training data
X_unseen_scaled = pd.DataFrame(scaler.transform(X_unseen), columns=X_unseen.columns)

#Making predictions on the unseen dataset
y_unseen_pred = RF_best.predict(X_unseen_scaled)

#Calculating evaluation metrics for the unseen set
mse_unseen = mean_squared_error(y_unseen, y_unseen_pred)  
r2_unseen = r2_score(y_unseen, y_unseen_pred)             

#Printing model performance on unseen data
print(f'Unseen Mean Squared Error: {mse_unseen:.4f}')
print(f'Unseen R-squared: {r2_unseen:.4f}')


In [ ]:
#Defining a function to log model evaluation metrics and hyperparameters to an Excel-file of chosen name
def log_results(model_name, params, mse_train, r2_train, mse_test, r2_test, mse_unseen, r2_unseen, filename="Results.xlsx"):
    """ Logs model results in separate sheets within the same Excel file, ensuring appending works correctly. """

    #Creating DataFrame for this model run
    result = pd.DataFrame([{
        **params,  
        "MSE_Train": mse_train,
        "R2_Train": r2_train,
        "MSE_Test": mse_test,
        "R2_Test": r2_test,
        "MSE_Unseen": mse_unseen,
        "R2_Unseen": r2_unseen
    }])

    #Introducing a short delay to avoid file conflicts if running in multiple notebooks
    time.sleep(1)

    #Checking if the file exists
    file_exists = os.path.exists(filename)

    if not file_exists:
        #Creating a new if file does not exist
        with pd.ExcelWriter(filename, engine="openpyxl", mode="w") as writer:
            result.to_excel(writer, sheet_name=model_name, index=False)
    else:
        #If file exists, loading it before appending
        try:
            with pd.ExcelWriter(filename, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
                #Reading existing sheet 
                try:
                    existing_df = pd.read_excel(filename, sheet_name=model_name, engine="openpyxl")
                    df_combined = pd.concat([existing_df, result], ignore_index=True)
                except (FileNotFoundError, ValueError):
                    df_combined = result  #If sheet does not exist, creating it

                #Saving results, ensuring correct appending
                df_combined.to_excel(writer, sheet_name=model_name, index=False)

        except PermissionError:
            #Printing error to warn user
            print(f"Error: Close the Excel file ({filename}) before running the script again.")

    #Printing a confirmation to ensure user results are logged
    print(f"Logged results for {model_name}: Train MSE={mse_train:.4f}, Test MSE={mse_test:.4f}, Unseen MSE={mse_unseen:.4f}")


In [ ]:
#Using the function to log results from RF regressor model
log_results(
    model_name="RF_reg_stat",
    params=best_params,
    mse_train=mse_train,  
    r2_train=r2_train,
    mse_test=mse_test,
    r2_test=r2_test,
    mse_unseen=mse_unseen,
    r2_unseen=r2_unseen
)